## 1 - Import packages


**Exercise**: Please mount your Google drive, and set up your working folder here.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
# start your code here
os.chdir("/content/drive/MyDrive/deep learning/Homework6") # change your working folder here

# end your code here

In [3]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(123)
tf.random.set_seed(123)



# 2 - Problem statement: SIGNS Dataset

One afternoon, with some friends we decided to teach our computers to decipher sign language. We spent a few hours taking pictures in front of a white wall and came up with the following dataset. It's now your job to build an algorithm that would facilitate communications from a speech-impaired person to someone who doesn't understand sign language.

- **Training set**: 1080 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (180 pictures per number).
- **Test set**: 120 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (20 pictures per number).

Note that this is a subset of the SIGNS dataset. The complete dataset contains many more signs.

Here are examples for each number, and how an explanation of how we represent the labels. These are the original pictures, before we lowered the image resolutoion to 64 by 64 pixels.
<figure>
<center>
<img src='https://drive.google.com/uc?id=1Nuox7mBJMGQt_QXZxmYGkXFEO5mjPf-B' />
<figcaption>Image Caption</figcaption></center>
</figure>


Run the following code to load the dataset.

In [4]:
# Loading the dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

Change the index below and run the cell to visualize some examples in the dataset.

In [5]:
# Example of a picture
index = 1
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

y = 0


# 3. Using One Hot encodings

Many times in deep learning you will have a y vector with numbers ranging from 0 to C-1, where C is the number of classes. If C is for example 4, then you might have the following y vector which you will need to convert as follows:
<figure>
<center>
<img src='https://drive.google.com/uc?id=1P02l9Fu2flQp0gsRl2bTAm0bsZ2cIMWI' />
<figcaption>
</figure>



This is called a "one hot" encoding, because in the converted representation exactly one element of each column is "hot" (meaning set to 1). In tensorflow, we can tranform the labels of samples to one hot vectors by calling

```
tf.keras.utils.to_categorical(y)
```
Please transform the lables (or the output of the dataset) to one hot vectors.
 

As usual you flatten the image dataset, then normalize it by dividing by 255. On top of that, you will convert each label to a one-hot vector . Run the cell below to do so.

In [6]:
# Flatten the training and test images
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1)
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1)
# Normalize image vectors
X_train = X_train_flatten/255.
X_test = X_test_flatten/255.
# Flatten the training and test labels
Y_train_flatten=Y_train_orig.flatten()
Y_test_flatten=Y_test_orig.flatten()

# Convert training labels (Y_train_flatten) and test labels (Y_test_flatten) to one hot matrices\
### START CODE HERE ###'
Y_train = tf.keras.utils.to_categorical(Y_train_flatten)
Y_test =tf.keras.utils.to_categorical(Y_test_flatten)
### END CODE HERE ###

print ("number of training examples = " + str(X_train.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 12288
number of test examples = 12288
X_train shape: (1080, 12288)
Y_train shape: (1080, 6)
X_test shape: (120, 12288)
Y_test shape: (120, 6)


**Note** that 12288 comes from $64 \times 64 \times 3$. Each image is square, 64 by 64 pixels, and 3 is for the RGB colors. Please make sure all these shapes make sense to you before continuing.

#4.Building your model

**Your goal** is to build an model capable of recognizing a sign with high accuracy. To do so, you are going to build a tensorflow model that is almost the same as one you have previously built for cat recognition and multi-class classification problems. If you already forget previous homeworks, please go back to review them to get some clues. 

**Previous related homeworks** 

"Planar_data_tensorflow.ipynb" in Homework 2

"deepNN_tensorflow.ipynb" in Homework 3

"regularized_deepNN_tensorflow.ipynb" in Homework 4

"batch-size-test-student-version.ipynb" In Homework 5

In [90]:
### START CODE HERE ###
model=tf.keras.Sequential([
    #Input layer                      
    tf.keras.layers.Input(shape=(12288)), # Set how many elements are there in every training image?

    #Start adding hidden layers  
    tf.keras.layers.Dense(50, activation= 'relu'), # Please set your neuron numbers, activation function and so on. 

    #please feel free to add more hidden layers here. 
     tf.keras.layers.Dense(45, activation= 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
     tf.keras.layers.Dense(35, activation= 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
     tf.keras.layers.Dense(20,activation='selu',kernel_regularizer=tf.keras.regularizers.L2(0.01)),

    # End adding hidden layers 

    #output layer                   
    tf.keras.layers.Dense(6, activation='softmax') # Set how many elements are there in every training output
            
])
### END CODE HERE ###'
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 50)                614450    
                                                                 
 dense_49 (Dense)            (None, 45)                2295      
                                                                 
 dense_50 (Dense)            (None, 35)                1610      
                                                                 
 dense_51 (Dense)            (None, 20)                720       
                                                                 
 dense_52 (Dense)            (None, 6)                 126       
                                                                 
Total params: 619,201
Trainable params: 619,201
Non-trainable params: 0
_________________________________________________________________


In [99]:
### START CODE HERE ###'
model.compile(optimizer= tf.keras.optimizers.Adamax(learning_rate=0.01),  # Optimizer. Please change 'None' to some meaningful learning rate which is usually <1
              loss=tf.keras.losses.CategoricalCrossentropy(
    from_logits=False, label_smoothing=0.0, axis=-1,
    name='categorical_crossentropy'
),
  # loss. Please change 'None' to some meaningful loss
              # List of metrics to monitor
              metrics=['accuracy'],
                 )
### END CODE HERE ###'

In [100]:
### START CODE HERE ###'
history = model.fit(
    X_train , #  training input data
    Y_train, # training output data
    batch_size=30, # how many samples (points) do you want to use to update weights once? 
    epochs=500 # how many iterations do you want to train your model? Your data will be reused to train the model for the number of epochs. I trained 5 min for this model
)
### END CODE HERE ###'

Epoch 1/500
36/36 [==============================] - 1s 5ms/step - loss: 1.8475 - accuracy: 0.1546
Epoch 2/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8062 - accuracy: 0.1685
Epoch 3/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8015 - accuracy: 0.1815
Epoch 4/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8096 - accuracy: 0.1602
Epoch 5/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8079 - accuracy: 0.1648
Epoch 6/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8033 - accuracy: 0.1667
Epoch 7/500
36/36 [==============================] - 0s 5ms/step - loss: 1.7998 - accuracy: 0.1620
Epoch 8/500
36/36 [==============================] - 0s 5ms/step - loss: 1.7982 - accuracy: 0.1852
Epoch 9/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8044 - accuracy: 0.1657
Epoch 10/500
36/36 [==============================] - 0s 5ms/step - loss: 1.8039 - accuracy: 0.1407
Epoch 11/


**If the training works, congratulations! You have built a model for recognizing SIGN language.**

In [96]:
model.evaluate(X_test,Y_test)

4/4 [==============================] - 0s 5ms/step - loss: 0.7260 - accuracy: 0.8417


[0.7260007262229919, 0.8416666388511658]

# 5. Grading rule

***Try your best to achieve high accuarcy.  The grade for this homework will based on your model final training accurracy and test accuracy. ***

For example, if your final training accurracy is 87.8 (consider one decimal) and test accuracy is 70.9, the total grade is 8.78 + 7.09 =15.87. The full credit is 10+10 =20.  It is hard to get full credit in this homework, please try your best.  





**Tips**

You can try to improve its accuracy by:
1. tuning the hyperparameters (learning rate, batch size, epoches and so on)
2. adjusting your layers numbers or nuron numbers.
3. changing optimizer or loss function
4. using dropout layer
5. add regularizers.L2
6. .....

Once again, here's a thumbs up for your work!

In [49]:
%pylab inline
import matplotlib.image as mpimg
img = mpimg.imread('images/thumbs_up.jpg')
imgplot = plt.imshow(img)
plt.show()

Populating the interactive namespace from numpy and matplotlib
